In [18]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys, h5py, time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mat
import seaborn as sb
import tensorflow as tf
import scipy

import sys
sys.path.append('../../../..')
import mutagenesisfunctions as mf
import bpdev as bd
import helper
from deepomics import neuralnetwork as nn
from deepomics import utils, fit, visualize, saliency

import contacts

from Bio import AlignIO
import time as time
import pandas as pd
np.random.seed(42)

In [19]:
def compare_EC(fam, comp='cWW_int', min_dist=8.0):

    ######### EXTRACT DATA ###########

    #extract SoM results after an APC correction
    arrayspath = '../../marks/Arrays/%s_mlp_%s_red.npy'%(fam,numhidden)
    hol_mut2 = np.load(arrayspath)
    seqlen,_, dims,_ = hol_mut2.shape
    C = bd.get_wc(arrayspath, seqlen, dims, bpugSQ=0, denoise='APC')
    
    #get coordinates for contact plot
    C[np.tril_indices(seqlen)] = np.min(C)
    bp_stretch = np.ravel(C)
    minval = np.sort(bp_stretch)[::-1][-1] #just gets the minimum value - can be changed to a different idx
    bs = np.where(C > minval)
    idx = np.argsort(C[bs])
    x = bs[0][idx] #sorts the x and y coords in order from lowest to highest so highest scores will be plotted last
    y = bs[1][idx]

    score_col = [C[xx,yy] for xx,yy in zip(x,y)] #gets the scores for each coordinate in the needed order

    #Load in the EC annotation
    ECannotfile = '../../marks/%s/%s.EC.interaction.txt'%(fam,fam) 
    EC_df = pd.read_csv(ECannotfile, delimiter='\s+')

    ######### QUALITATIVE CONTACTS ###########

    #All cWW annotated interactions
    cWW_int = EC_df[EC_df['interactions'] == 'cWW']
    #All PDB contacts < 8 angstroms
    close_cont = EC_df[EC_df['minimum_atom_distance'] <= min_dist] 
    

    ######### QUANTITATIVE GRAPHS ###########

    #Get the real positive and negative pdb contacts
    def groundtruth(comp=comp):
        
        if comp == 'cWW_int':
            real_pos = np.zeros((seqlen,seqlen))
            for ii in range(len(cWW_int)):
                real_pos[cWW_int.iloc[ii, 1], cWW_int.iloc[ii, 2]] = 1.
            real_neg = np.ones((seqlen,seqlen))
            for ii in range(len(cWW_int)):
                real_neg[cWW_int.iloc[ii, 1], cWW_int.iloc[ii, 2]] = 0.
        return (real_pos, real_neg)
    
    def sumstats(thresh):
        #Get the top thresh SoM scores
        som_pos = bd.plot_wcrank(C, seqlen, thresh)#*2) #because of how we reset C above, we can just use the threshold because both*2)
        som_pos[np.tril_indices(seqlen)] = 0.
        #Get the bottom thresh SoM scores
        som_neg = (~som_pos.astype('bool'))*1 #incantation converts binary to boolean, flips them then back to binary

        TP = np.sum(real_pos*som_pos)
        FP = np.sum(real_neg*som_pos)
        TN = np.sum(real_neg*som_neg)
        FN = np.sum(real_pos*som_neg)

        PPV = TP/(TP+FP)
        FDR = FP/(TP+FP)
        TPR = TP/(TP+FN)

        return (PPV, FDR, TPR, TP, FP, TN, FN)

    def sumstats_EC(thresh):
        #Top thresh EC scores
        n_EC_P = len(EC_df.iloc[:thresh, :])
        EC_pos = np.zeros((seqlen,seqlen))
        for ii in range(n_EC_P):
            EC_pos[EC_df.iloc[ii, 1], EC_df.iloc[ii, 2]] = 1.
        EC_neg = np.ones((seqlen,seqlen))
        for ii in range(n_EC_P):
            EC_neg[EC_df.iloc[ii, 1], EC_df.iloc[ii, 2]] = 0.

        TP = np.sum(real_pos*EC_pos)
        FP = np.sum(real_neg*EC_pos)
        TN = np.sum(real_neg*EC_neg)
        FN = np.sum(real_pos*EC_neg)

        PPV = TP/(TP+FP)
        FDR = FP/(TP+FP)
        TPR = TP/(TP+FN)

        return (PPV, FDR, TPR, TP, FP, TN, FN)
    
    real_pos, real_neg = groundtruth(comp=comp)
    extent = len(cWW_int)

    EC_ppv = []
    EC_tpr = []
    SoM_ppv = []
    SoM_tpr = []
    for tr in range(extent):
        ecst = sumstats_EC(tr)
        somst = sumstats(tr)

        EC_ppv.append(ecst[0])
        EC_tpr.append(ecst[2])

        SoM_ppv.append(somst[0])
        SoM_tpr.append(somst[2]) 

        
    #get the colour map to select colors for plotting
    pal = 'Blues'
    cmap = mat.cm.get_cmap(pal)


    markersize = 250
    fig = plt.figure(figsize=(30,15))
    
    #plot the ranked PPV
    ax1 = fig.add_subplot(1,2,1)
    ax1.plot(range(extent),SoM_ppv, '-b', label='SoM', linewidth=7.)
    ax1.plot(range(extent),EC_ppv, '-r', label='EC', linewidth=7.)
    ax1.set_xlabel('Number of contacts predicted', fontsize=30)
    ax1.set_ylabel('Positive Predictive Value', fontsize=30)
    ax1.set_title('cWW interactions predicted', fontsize=30)
    ax1.legend(loc='best', fontsize=20, bbox_to_anchor=(1., 0.5))
    ax1.tick_params(axis='both', which='major', labelsize=30)
    ax1.set_ylim([0.,1.])
    
    ax2 = fig.add_subplot(1,2,2)
    ax2.scatter(x, y, c=score_col, cmap=pal, label = 'SoM', vmin=0., s=markersize) #plot SoM results
    ax2.scatter(EC_df.iloc[:,2][::-1], EC_df.iloc[:,1][::-1], c=EC_df['score'][::-1], cmap='Reds', label = 'EC', s=markersize)
    
    #ax2.scatter(cWW_int.iloc[:,2], cWW_int.iloc[:,1], label = 'cWW',
     #           edgecolors='k',  marker='o', facecolors='none', linewidths=1.5, s=markersize)
    #ax2.scatter(cWW_int.iloc[:,1], cWW_int.iloc[:,2],
     #           edgecolors='k',  marker='o', facecolors='none', linewidths=1.5, s=markersize)
    
    ax2.set_title('%s: SoM vs. EC '%(fam), fontsize=30)
    ax2.legend(labels=['SoM', 'EC', 'cWW', ''], loc='best', fontsize=20, bbox_to_anchor=(1., 0.5))
    ax2.set_xlabel('Sequence Position', fontsize=30)
    ax2.set_ylabel('Sequence Position', fontsize=30)
    ax2.invert_yaxis()
    ax2.set_facecolor('w')#(cmap(0.))
    ax2.tick_params(axis='both', which='major', labelsize=30)

    plt.savefig('../../marks/Stats_graphs/cWW_int/extcWW/%s_cWW_int_excWW_stats.png'%(fam))
    plt.savefig('../../marks/%s/Images_mlp/%s_cWW_int_excWW_stats.png'%(fam,fam))
    plt.close()

In [20]:
families = ['RF00002', 'RF00005', 'RF00010', 'RF00017', 'RF00023', 'RF00050',
            'RF00059', 'RF00162', 'RF00167', 'RF00169', 'RF00174', 'RF00234',
            'RF00380', 'RF00504', 'RF01734', 'RF01786', 'RF01831', 'RF01852', 'RF02001']
numhidden =512

In [21]:
for i,fam in enumerate(families):
    bars = '='*i
    space = ' '*(len(families)-i)
    sys.stdout.write('\r[%s] --- %s'%(bars+space, fam))
    compare_EC(fam)
sys.stdout.write('\r[%s] ------ DONE'%('='*len(families)))
    
    

[                   ] --- RF00002

/Users/steffanpaul/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:81: RuntimeWarning: invalid value encountered in double_scalars
/Users/steffanpaul/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:82: RuntimeWarning: invalid value encountered in double_scalars
/Users/steffanpaul/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in double_scalars
/Users/steffanpaul/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in double_scalars


[===================] ------ DONE